In [53]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from psycopg2.extras import execute_values
from torch.utils.data import DataLoader
import psycopg2
import pandas as pd
import random
import numpy as np
import faiss

## Finetune sbert

In [2]:
connection = psycopg2.connect(user="postgres", password="postgres", host="127.0.0.1", port="63333", database="stack_overflow")

In [29]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""select q.body_text, q.title, sq.body_text, sq.title
        FROM posts_19 q
        JOIN posts_19_tags pt on pt.post_id=q.id
        JOIN tags t on pt.tag_id=t.id
        JOIN posts_19 q_aa ON q_aa.id = q.accepted_answer_id
        JOIN posts_19 sa ON sa.owner_user_id=q_aa.owner_user_id
        JOIN posts_19 sq ON sq.id=sa.parent_id
        WHERE q.body_text is not null
        and sa.parent_id != q.id 
        and q.post_type_id=1 
        and q.creation_date < '2022-01-01'
        and t.tag_name = 'python'
        and q_aa.post_type_id=2 
        and sa.post_type_id=2
        limit 10000""")
        posts = cursor.fetchall()

In [33]:
train_examples = [] 
for q_text, q_title, sq_text, sq_title in random.sample(posts, 10):
    post_text = q_text
    if q_title is not None:
        post_text = q_title + "\n\n" + q_text
    sim_post_text = sq_text
    if sq_title is not None:
        sim_post_text = sq_title + "\n\n" + sq_text
    train_examples.append(InputExample(texts=[post_text, sim_post_text], label=.98))

In [34]:
train_dataloader = DataLoader(train_examples, batch_size=2)

model = SentenceTransformer('nli-distilroberta-base-v2')

train_loss = losses.CosineSimilarityLoss(model=model)

In [35]:
num_epochs = 3
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_epochs, warmup_steps=warmup_steps, show_progress_bar=True)

Epoch: 100%|██████████| 3/3 [00:17<00:00,  5.86s/it]


## Test recall

In [36]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""select p.id, p.title, p.body_text
        from posts_19 p
        join posts_19_tags pt on pt.post_id=p.id
        join tags t on pt.tag_id=t.id
        where p.body_text is not null
        and p.post_type_id=1 
        and creation_date < '2022-01-01'
        and t.tag_name = 'python'
        limit 10000""")
        all_posts = cursor.fetchall()

In [37]:
posts_texts = []
posts_ids = []
for p_id, title, body in all_posts:
    post_text = body
    if title is not None:
        post_text = title + "\n\n" + body
    posts_ids.append(p_id)
    posts_texts.append(post_text)

In [38]:
encoded_data = model.encode(posts_texts)

In [41]:
vectors_ids = np.array(posts_ids)

In [42]:
v_id2idx = {v_id: idx for idx, v_id in enumerate(vectors_ids)}

In [40]:
encoded_data.shape

(10000, 768)

In [45]:
index = faiss.index_factory(768, "IDMap,Flat")
index.train(encoded_data)
index.add_with_ids(encoded_data, vectors_ids)

## Test recall

In [46]:
MIN_ANSWERS = 9
with connection:
    with connection.cursor() as cursor:
        cursor.execute(f"""select p.id, p.title, p.body_text
        from posts_19 p
        join posts_19_tags pt on pt.post_id=p.id
        join posts_19 a on a.parent_id=p.id
        join tags t on pt.tag_id=t.id
        where p.body_text is not null
        and p.creation_date < '2022-01-01'
        and t.tag_name = 'python'
        group by p.id
        HAVING count(*) > {MIN_ANSWERS}
        limit 30""")
        recall_posts = cursor.fetchall()

In [47]:
posts_vectors=[]
posts_ids = []
for p_id, title, body in recall_posts:
    posts_ids.append(p_id)
    post_text = body
    if title is not None:
        post_text = title + "\n\n" + body
    posts_vectors.append(post_text)
posts_vectors = model.encode(posts_vectors)

In [48]:
N_SIMILAR_QUESTIONS = 500
distances, similar_questions = index.search(posts_vectors, N_SIMILAR_QUESTIONS)
similar_questions.shape

(30, 500)

In [52]:
missing_vectors = 0
features_df = []
exact_distances = []
for p_id, p_vector, similar_qs in zip(posts_ids, posts_vectors, similar_questions):
    for idx, s_q in enumerate(similar_qs):
        try:
            sq_vector = encoded_data[v_id2idx[s_q]]
            features_df.append([p_id, s_q, p_vector-sq_vector])
            exact_distances.append(np.linalg.norm(p_vector-sq_vector))
        except KeyError:
            missing_vectors +=1

features_df = pd.DataFrame(features_df, columns=["q_id", "sq_id", "X"])
features_df.head()

,q_id,sq_id,X
0,1854,3333243,"[0.3333763, 0.110242635, -0.11257695, -0.07789..."
1,1854,647515,"[0.24326734, -0.00912565, 0.10152519, -0.11551..."
2,1854,1405913,"[0.011253402, -0.20168363, 0.09048267, 0.06326..."
3,1854,680207,"[0.12657654, -0.029177487, 0.10178462, 0.05899..."
4,1854,3518491,"[0.08814222, 0.0036339462, 0.111369364, 0.1081..."


In [54]:
with connection:
    with connection.cursor() as cursor:
        execute_values(cursor, """select q_id, sq_id, q_a.owner_user_id, q_a.score
                                FROM (VALUES %s) AS base_questions (q_id, sq_id)
                                JOIN posts_19 q_a ON q_a.parent_id = q_id
                                JOIN posts_19 sq ON sq.id = sq_id
                                JOIN posts_19 sq_a ON sq_a.id = sq.accepted_answer_id
                                WHERE sq_a.owner_user_id=q_a.owner_user_id""",
                                features_df[["q_id", "sq_id"]].values.tolist(), page_size=len(features_df))
        q_ds = cursor.fetchall()
        q_ds = pd.DataFrame(q_ds, columns=["q_id", "sq_id", "user_id", "score"])
q_ds.head()

,q_id,sq_id,user_id,score
0,32899,1911281,3571,234
1,38987,595374,424499,184
2,42950,2545532,4279,20
3,42950,5802108,4279,20
4,42950,471928,4279,20


In [55]:
len(q_ds)

16